In [19]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from peft import PeftModel

model_path = "../model/classifier_model"

# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained(model_path)

# 베이스 모델 로드 (LoRA 적용 전 베이스 모델)
base_model_name = "klue/bert-base"
base_model = BertForSequenceClassification.from_pretrained(base_model_name, num_labels=5)

# LoRA 모델 로드
model = PeftModel.from_pretrained(base_model, model_path)

model.eval()  # 평가 모드로 변경

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = logits.argmax(dim=-1).item()
    
    return predicted_class_id

text = "이번 학기 수강신청 언제 시작하나요?"
pred_label = predict(text)
print("예측 라벨:", pred_label)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


예측 라벨: 2
